In [1]:
import tensorflow as tf
import pandas as pd
import talib


import yfinance as yf
import datetime as dt
import pandas_ta as pta
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv1D, Conv2D, Flatten, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [2]:
import sys

pth_apple = "/Users/josesanchez/Desktop/Code/GitHub/StockPrediction/FileBase"
pth_win = "/Users/Usuario/Desktop/GitHub/StockPrediction/FileBase"
sys.path.append(pth_win)


from root_functions2 import add_indicators, up_down, full_prediction, full_prediction_binary, result, aprox_beneficios

In [3]:
load_model = tf.keras.models.load_model('saved_model/my_model')

In [4]:
load_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 32)             3488      
_________________________________________________________________
lstm (LSTM)                  (None, 1, 128)            82432     
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 128)            512       
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 256)            394240    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 256)            1024      
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 128)            197120    
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 128)            5

In [5]:
sample_per = 5000
stock = 'EURGBP=X'

end = dt.date.today()
start = end - dt.timedelta(days=(sample_per))
interval = '1d'

In [6]:
database = yf.download(stock, start=start, end=end, interval=interval)

[*********************100%***********************]  1 of 1 completed


In [7]:
database

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-01-02,0.73540,0.74455,0.73530,0.74260,0.74260,0
2008-01-03,0.74250,0.74830,0.74140,0.74680,0.74680,0
2008-01-04,0.74685,0.74910,0.74240,0.74740,0.74740,0
2008-01-07,0.74780,0.74790,0.74400,0.74650,0.74650,0
2008-01-08,0.74650,0.74650,0.74200,0.74560,0.74560,0
...,...,...,...,...,...,...
2021-09-06,0.85762,0.85856,0.85660,0.85740,0.85740,0
2021-09-07,0.85794,0.86128,0.85750,0.85780,0.85780,0
2021-09-08,0.85944,0.86017,0.85770,0.85943,0.85943,0


In [8]:
#Añadimos indicadores

ema1=200
ema2=50
ema3=20

data_base = add_indicators(database)

#Añadimos columna up_down
data_base = up_down(data_base)

#Listado indicadores
data_base.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', '200ema',
       '50ema', '20ema', '14-high',
       ...
       'CDL_STICKSANDWICH', 'CDL_TAKURI', 'CDL_TASUKIGAP', 'CDL_THRUSTING',
       'CDL_TRISTAR', 'CDL_UNIQUE3RIVER', 'CDL_UPSIDEGAP2CROWS',
       'CDL_XSIDEGAP3METHODS', 'log_ret', 'Up_Down'],
      dtype='object', length=109)

In [9]:
data_base

,Open,High,Low,Close,Adj Close,Volume,200ema,50ema,20ema,14-high,...,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS,log_ret,Up_Down
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,0.73540,0.74455,0.73530,0.74260,0.74260,0,0.742600,0.742600,0.742600,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
2008-01-03,0.74250,0.74830,0.74140,0.74680,0.74680,0,0.744711,0.742765,0.743000,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005656,1
2008-01-04,0.74685,0.74910,0.74240,0.74740,0.74740,0,0.745616,0.742947,0.743419,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000803,1
2008-01-07,0.74780,0.74790,0.74400,0.74650,0.74650,0,0.745840,0.743086,0.743712,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.001204,0
2008-01-08,0.74650,0.74650,0.74200,0.74560,0.74560,0,0.745791,0.743184,0.743892,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.001206,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-06,0.85762,0.85856,0.85660,0.85740,0.85740,0,0.866480,0.855697,0.855768,0.86022,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.000827,0
2021-09-07,0.85794,0.86128,0.85750,0.85780,0.85780,0,0.866393,0.855780,0.855962,0.86128,...,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.000467,1
2021-09-08,0.85944,0.86017,0.85770,0.85943,0.85943,0,0.866324,0.855923,0.856292,0.86128,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001900,1


In [10]:
#Filtramos las columnas que nos interesan para obtener el dataset de trabajo
db = data_base.copy()


data = db[['log_ret','Open', 'High', 'Low', 'Close', 'Adj Close','Volume', '200ema',
       '50ema', '20ema', '14-high', '14-low', '%K', '%D', 'RSI', 'MACD',
       'MACDh', 'MACDs', 'CCI', 'PSAR', 'BBl', 'BBm', 'BBu', 'BBb', 'BBp',
       'ATR', 'ADX', 'ADX_DMP', 'ADX_DMN', 'APO', 'AROON_D', 'AROON_U',
       'AROON_OSC', 'BOP', 'CMO', 'MOM', 'PPO', 'PPO_h', 'PPO_s', 'ROC',
       'STOCHRSI_k', 'STOCHRSI_d', 'TRIX', 'TRIX_s', 'WILLR', 'NATR',
       'CDL_2CROWS', 'CDL_3BLACKCROWS', 'CDL_3INSIDE', 'CDL_3LINESTRIKE',
       'CDL_3OUTSIDE', 'CDL_3STARSINSOUTH', 'CDL_3WHITESOLDIERS',
       'CDL_ABANDONEDBABY', 'CDL_ADVANCEBLOCK', 'CDL_BELTHOLD',
       'CDL_BREAKAWAY', 'CDL_CLOSINGMARUBOZU', 'CDL_CONCEALBABYSWALL',
       'CDL_COUNTERATTACK', 'CDL_DARKCLOUDCOVER', 'CDL_DOJI_10_0.1',
       'CDL_DOJISTAR', 'CDL_DRAGONFLYDOJI', 'CDL_ENGULFING',
       'CDL_EVENINGDOJISTAR', 'CDL_EVENINGSTAR', 'CDL_GAPSIDESIDEWHITE',
       'CDL_GRAVESTONEDOJI', 'CDL_HAMMER', 'CDL_HANGINGMAN', 'CDL_HARAMI',
       'CDL_HARAMICROSS', 'CDL_HIGHWAVE', 'CDL_HIKKAKE', 'CDL_HIKKAKEMOD',
       'CDL_HOMINGPIGEON', 'CDL_IDENTICAL3CROWS', 'CDL_INNECK',
       'CDL_INSIDE', 'CDL_INVERTEDHAMMER', 'CDL_KICKING',
       'CDL_KICKINGBYLENGTH', 'CDL_LADDERBOTTOM', 'CDL_LONGLEGGEDDOJI',
       'CDL_LONGLINE', 'CDL_MARUBOZU', 'CDL_MATCHINGLOW', 'CDL_MATHOLD',
       'CDL_MORNINGDOJISTAR', 'CDL_MORNINGSTAR', 'CDL_ONNECK',
       'CDL_PIERCING', 'CDL_RICKSHAWMAN', 'CDL_RISEFALL3METHODS',
       'CDL_SEPARATINGLINES', 'CDL_SHOOTINGSTAR', 'CDL_SHORTLINE',
       'CDL_SPINNINGTOP', 'CDL_STALLEDPATTERN', 'CDL_STICKSANDWICH',
       'CDL_TAKURI', 'CDL_TASUKIGAP', 'CDL_THRUSTING', 'CDL_TRISTAR',
       'CDL_UNIQUE3RIVER', 'CDL_UPSIDEGAP2CROWS', 'CDL_XSIDEGAP3METHODS']]

In [11]:
data

,log_ret,Open,High,Low,Close,Adj Close,Volume,200ema,50ema,20ema,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,NaN,0.73540,0.74455,0.73530,0.74260,0.74260,0,0.742600,0.742600,0.742600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-01-03,0.005656,0.74250,0.74830,0.74140,0.74680,0.74680,0,0.744711,0.742765,0.743000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-01-04,0.000803,0.74685,0.74910,0.74240,0.74740,0.74740,0,0.745616,0.742947,0.743419,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-01-07,-0.001204,0.74780,0.74790,0.74400,0.74650,0.74650,0,0.745840,0.743086,0.743712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-01-08,-0.001206,0.74650,0.74650,0.74200,0.74560,0.74560,0,0.745791,0.743184,0.743892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-06,-0.000827,0.85762,0.85856,0.85660,0.85740,0.85740,0,0.866480,0.855697,0.855768,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-09-07,0.000467,0.85794,0.86128,0.85750,0.85780,0.85780,0,0.866393,0.855780,0.855962,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0
2021-09-08,0.001900,0.85944,0.86017,0.85770,0.85943,0.85943,0,0.866324,0.855923,0.856292,...,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
data.reset_index(inplace=True)
data.drop(columns=data.columns[0], inplace=True)

C:\Users\Usuario\anaconda3\envs\StockPrediction\lib\site-packages\pandas\core\frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
data

,log_ret,Open,High,Low,Close,Adj Close,Volume,200ema,50ema,20ema,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
0,NaN,0.73540,0.74455,0.73530,0.74260,0.74260,0,0.742600,0.742600,0.742600,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.005656,0.74250,0.74830,0.74140,0.74680,0.74680,0,0.744711,0.742765,0.743000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000803,0.74685,0.74910,0.74240,0.74740,0.74740,0,0.745616,0.742947,0.743419,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.001204,0.74780,0.74790,0.74400,0.74650,0.74650,0,0.745840,0.743086,0.743712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.001206,0.74650,0.74650,0.74200,0.74560,0.74560,0,0.745791,0.743184,0.743892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3540,-0.000827,0.85762,0.85856,0.85660,0.85740,0.85740,0,0.866480,0.855697,0.855768,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3541,0.000467,0.85794,0.86128,0.85750,0.85780,0.85780,0,0.866393,0.855780,0.855962,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0
3542,0.001900,0.85944,0.86017,0.85770,0.85943,0.85943,0,0.866324,0.855923,0.856292,...,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3543,-0.001152,0.85840,0.85880,0.85245,0.85844,0.85844,0,0.866246,0.856022,0.856497,...,100.0,0.0,0.0,0.0,0.0,0.0,-100.0,0.0,0.0,0.0


In [63]:
data = data.iloc[1:,:]

In [64]:
#Convertimos todas las columnas a valor numerico
data = data.apply(pd.to_numeric)

In [65]:
data.dropna(axis='rows')

,log_ret,Open,High,Low,Close,Adj Close,Volume,200ema,50ema,20ema,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
1,0.005656,0.74250,0.74830,0.74140,0.74680,0.74680,0,0.744711,0.742765,0.743000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000803,0.74685,0.74910,0.74240,0.74740,0.74740,0,0.745616,0.742947,0.743419,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.001204,0.74780,0.74790,0.74400,0.74650,0.74650,0,0.745840,0.743086,0.743712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.001206,0.74650,0.74650,0.74200,0.74560,0.74560,0,0.745791,0.743184,0.743892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.004426,0.74570,0.75010,0.74450,0.74890,0.74890,0,0.746322,0.743409,0.744369,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3540,-0.000827,0.85762,0.85856,0.85660,0.85740,0.85740,0,0.866480,0.855697,0.855768,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3541,0.000467,0.85794,0.86128,0.85750,0.85780,0.85780,0,0.866393,0.855780,0.855962,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0
3542,0.001900,0.85944,0.86017,0.85770,0.85943,0.85943,0,0.866324,0.855923,0.856292,...,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3543,-0.001152,0.85840,0.85880,0.85245,0.85844,0.85844,0,0.866246,0.856022,0.856497,...,100.0,0.0,0.0,0.0,0.0,0.0,-100.0,0.0,0.0,0.0


In [66]:
data

,log_ret,Open,High,Low,Close,Adj Close,Volume,200ema,50ema,20ema,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
1,0.005656,0.74250,0.74830,0.74140,0.74680,0.74680,0,0.744711,0.742765,0.743000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000803,0.74685,0.74910,0.74240,0.74740,0.74740,0,0.745616,0.742947,0.743419,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.001204,0.74780,0.74790,0.74400,0.74650,0.74650,0,0.745840,0.743086,0.743712,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.001206,0.74650,0.74650,0.74200,0.74560,0.74560,0,0.745791,0.743184,0.743892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.004426,0.74570,0.75010,0.74450,0.74890,0.74890,0,0.746322,0.743409,0.744369,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3540,-0.000827,0.85762,0.85856,0.85660,0.85740,0.85740,0,0.866480,0.855697,0.855768,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3541,0.000467,0.85794,0.86128,0.85750,0.85780,0.85780,0,0.866393,0.855780,0.855962,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0
3542,0.001900,0.85944,0.86017,0.85770,0.85943,0.85943,0,0.866324,0.855923,0.856292,...,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3543,-0.001152,0.85840,0.85880,0.85245,0.85844,0.85844,0,0.866246,0.856022,0.856497,...,100.0,0.0,0.0,0.0,0.0,0.0,-100.0,0.0,0.0,0.0


In [28]:



#Escalamos los valores con un MinMaxScaler

#data['200ema'] = data['200ema'].pct_change()
#data['50ema'] = data['50ema'].pct_change()
#data['20ema'] = data['20ema'].pct_change()

#data.dropna(inplace=True)

values = data.values

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

values = scaled

C:\Users\Usuario\anaconda3\envs\StockPrediction\lib\site-packages\sklearn\preprocessing\_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Usuario\anaconda3\envs\StockPrediction\lib\site-packages\sklearn\preprocessing\_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [26]:
data

,log_ret,Open,High,Low,Close,Adj Close,Volume,200ema,50ema,20ema,...,CDL_SPINNINGTOP,CDL_STALLEDPATTERN,CDL_STICKSANDWICH,CDL_TAKURI,CDL_TASUKIGAP,CDL_THRUSTING,CDL_TRISTAR,CDL_UNIQUE3RIVER,CDL_UPSIDEGAP2CROWS,CDL_XSIDEGAP3METHODS
0,NaN,0.86307,0.86378,0.86000,0.86297,0.86297,0,0.862970,0.862970,0.862970,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.001669,0.86149,0.86190,0.85862,0.86153,0.86153,0,0.862246,0.862914,0.862833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.001892,0.85972,0.86001,0.85660,0.85990,0.85990,0,0.861456,0.862795,0.862554,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.001047,0.85897,0.86124,0.85780,0.85900,0.85900,0,0.860833,0.862647,0.862215,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.001024,0.85981,0.86259,0.85950,0.85988,0.85988,0,0.860639,0.862538,0.861993,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,-0.000827,0.85762,0.85856,0.85660,0.85740,0.85740,0,0.855649,0.855645,0.855769,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,0.000467,0.85794,0.86128,0.85750,0.85780,0.85780,0,0.855691,0.855730,0.855963,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0
70,0.001900,0.85944,0.86017,0.85770,0.85943,0.85943,0,0.855764,0.855875,0.856293,...,-100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,-0.001152,0.85840,0.85880,0.85245,0.85844,0.85844,0,0.855816,0.855976,0.856497,...,100.0,0.0,0.0,0.0,0.0,0.0,-100.0,0.0,0.0,0.0
